# Filtro de tipo de tendencia: ADX + DMI

Funcionalidad que permite determinar la existencia de tendencia y la fortaleza de la misma. Si hay tendencia se recomienda usar indicadores de tipo trend following (si hay resistencias - máximos anteriores - informarlo con alertas). Si no hay tendencia, se recomienda usar osciladores contrarian.

El ADX (Average Directional Index) es un indicador de análisis técnico, utilizado para conocer si los precios se encuentran en tendencia o en rango y para medir la fuerza de la tendencia.

Cuando el ADX es mayor que 30, el mercado se encuentra en una tendencia fuerte, cuando está entre 20 y 30 no está bien definido y cuando es menor a 20 indica que el mercado está en rango (sin tendencia)

- Una de las herramientas más utilizadas para este fin es el indicador ADX. De hecho, el ADX es el indicador que se suele usar para confirmar, en última instancia, la formación de la tendencia de un mercado. 
- el indicador ADX será el encargado de asegurar al operador que el movimiento en el precio del valor se mantendrá por cierto plazo en la misma dirección, gracias a la fortaleza de la tendencia.
- el ADX no identifica si una tendencia es alcista o bajista. Lo que determina el ADX es la fortaleza de una tendencia.
- Es necesario el complemento con otro indicador que termine de verificar la dirección de la tendencia.
- el ADX se representa como una línea cuyos valores oscilan entre 0 y 100, e indica la fortaleza de una tendencia en el valor de un activo. Cuanto mayor sea el valor que indica el ADX, mayor será la fortaleza de la tendencia del precio, que puede ser tanto alcista como bajista. De la misma manera, valores bajos indican tendencias débiles o incluso ausencia de tendencia.
- Valores de referencia:
        Valor del ADX	Fortaleza de latendencia
        0-25	Ausencia de tendencia
        25-50	Tendencia fuerte
        50-75	Tendencia muy fuerte
        75-100	Tendencia extremadamente fuerte
- al ADX se lo suele complementar con dos líneas en su gráfico. Estas son el indicador direccional positivo (+DI) y el indicador direccional negativo (-DI). Los indicadores direccionales también muestran la fortaleza de una tendencia. Por lo que, probabilísticamente, la dirección estará dada por cuál de los dos presenta un valor superior.
- al detectar un aumento en el valor del ADX, un trader querrá saber qué dirección tomará la tendencia, para actuar al respecto. Entonces, mirará la evolución de las líneas de los indicadores. Si el indicador direccional positivo (+DI) se encuentra por encima del negativo (-DI), esto significará que la fortaleza de una tendencia alcista supera a la de una tendencia bajista. Por lo tanto, el operador podrá suponer que la tendencia será alcista. En caso contrario, se da de igual manera. Una tendencia bajista podrá ser anticipada por el trader si observa que, al aumentar la fortaleza del ADX, indicando una tendencia futura, el indicador direccional negativo (-DI) supera la positivo (+DI). Esto se ve en el siguiente gráfico.
- Si la rotura del rango coincide con una subida del ADX de menos de 25 a más de 25, el movimiento del precio es lo suficientemente fuerte como para continuar en la dirección en la que se produce la ruptura. Por eso es importante el indicador ADX. 

Fuente:
https://www.avatrade.es/educacion/trading-para-principiantes/indicador-adx

# INDICE DE FUNCIONES:
- filterTrend(activos, timeframe, report = False):
    Esta función utiliza el indicador técnico ADX para determinar la fortaleza de la tendencia. Adicionalmente se informa el sentido
    de la tendencia según el sentido de los cruces de los DMI- y DMI+. Si DMI+ > DMI - => Tendencia Alcista, sino Bajista.
    Se utilizan los siguientes rangos: 
        - Sin tendencia:                    0  >= ADX < 25
        - tendencia Fuerte:                 25 >= ADX < 50
        - tendencia Muy Fuerte              50 >= ADX < 75
        - tendencia Extremadamente fuerte   75 >= ADX < 100
- getDMISignals(activos, timeframe):
    Esta función devuelve los cruces vigentes de los indicadores DMI+ y DMI- del listado de activos
    solicitado, así como el sentido del cruce, la fecha de la señal, el precio de la señal, el rendimiento acumulado a la fecha
    y el rendimiento acumulado, todo ordenado en forma descendente por cantidad de días transcurridos desde la señal.

In [2]:
# FUNCIONES NECESARIAS PARA OBTENER DATA FINANCIERA:

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time

def getDataYf(ticker, tipo, interval, data_from = None, data_to = None, period = None):
    """
    Es una función para descargar market data de Yahoo Finance con la librería yfinance.
    
    ## Inputs:
        >ticker: el nombre del ticker.
        >tipo: si es "no end" no se indica hasta cuándo (data_to), se obtiene hasta el último día disponible. Si es "end" es
        necesario indicar hasta cuánto (data_to). En ambos casos hay que indicar desde qué fecha (data_from). Si es "period" 
        no se indica ni desde cuándo ni hasta cuándo, sólo el argumento "period" con la cantidad de tiempo a obtener.
        >now : si es True, no se indica hasta cuándo (data_to), se obtiene hasta el último día disponible. Si se indica False, es
        necesario indicar hasta cuánto (data_to).
        >interval: el timeframe (ej. 1mo, 1h, 1d, 1wk, etc)
        >data_from: data desde qué fecha.
        >data_to: data hasta qué fecha (no inclusive el día). Sólo es aplicable si now == True.
        >period : en caso de tipo = "period", se pasa este argumento que refiere a la cantidad de tiempo a obtener. Ej. 1y, 2y, 3y, etc.
        
    ## Outputs:
        >series OHLC ajustadas del ticker.
    """
    import yfinance as yf
    import pandas as pd
    
    if tipo == "no end":
        data = yf.download(ticker, start = data_from, interval = interval, progress = False, auto_adjust = True)
    elif tipo == "end":
        data = yf.download(ticker, start = data_from, end = data_to, interval = interval, progress = False, auto_adjust = True)
    elif tipo == "period":
        data = yf.download(ticker, interval = interval, period = period, progress = False, auto_adjust = True)
    return data



def getDataYfMulti(activos, tipo, interval, data_from = None, data_to = None, period = None, swap = True):
    """
    Función para hacer batch requests (varios tickers a la vez), que será la fx que más voy a utilizar para market data.
    
    ## Inputs:
        >tickers: es una lista con los tickers de los cuales se va a obtener market data.
        >tipo: si es "no end" no se indica hasta cuándo (data_to), se obtiene hasta el último día disponible. Si es "end" es
        necesario indicar hasta cuánto (data_to). En ambos casos hay que indicar desde qué fecha (data_from). Si es "period" 
        no se indica ni desde cuándo ni hasta cuándo, sólo el argumento "period" con la cantidad de tiempo a obtener.
        >now : si es True, no se indica hasta cuándo (data_to), se obtiene hasta el último día disponible. Si se indica False, es
        necesario indicar hasta cuánto (data_to).
        >interval: el timeframe (ej. 1mo, 1h, 1d, 1wk, etc)
        >data_from: data desde qué fecha.
        >data_to: data hasta qué fecha (no inclusive el día). Sólo es aplicable si now == True.
        >period : en caso de tipo = "period", se pasa este argumento que refiere a la cantidad de tiempo a obtener. Ej. 1y, 2y, 3y, etc.
        >swap : si es True, divide el df en tickers y cada uno tiene su OHLC. Si es False, tenemos cada columna OHLC y dentro todos los tickers.
        
    ## Outputs:
        >series OHLC ajustadas del ticker.
    """
    import yfinance as yf
    import pandas as pd
    
    lideres_arg = ["ALUA.BA", "BBAR.BA", "BMA.BA", "BYMA.BA", "CEPU.BA", "COME.BA", "CRES.BA", "CVH.BA", "EDN.BA", 
                   "GGAL.BA", "LOMA.BA", "MIRG.BA", "PAMP.BA", "SUPV.BA", "TECO2.BA", "TGNO4.BA", "TGSU2.BA", "TRAN.BA", 
                   "TXAR.BA", "VALO.BA", "YPFD.BA"]

    general_arg = ["AGRO.BA", "AUSO.BA", "BHIP.BA", "BOLT.BA", "BPAT.BA", "CADO.BA", "CAPX.BA", "CARC.BA", "CECO2.BA", 
                   "CELU.BA", "CGPA2.BA", "CTIO.BA", "DGCU2.BA", "FERR.BA", "FIPL.BA", "GAMI.BA", "GCDI.BA", "GCLA.BA", 
                   "HARG.BA", "HAVA.BA", "INVJ.BA", "IRSA.BA", "LEDE.BA", "LONG.BA", "METR.BA", "MOLA.BA", "MOLI.BA", 
                   "MORI.BA", "OEST.BA", "PATA.BA", "RICH.BA", "RIGO.BA", "SAMI.BA", "SEMI.BA"]

    cedears = ["AAL", "AAPL", "ABBV", "ABEV", "ABNB", "ABT", "ADBE", "ADGO", "ADI", "ADP", "AEM", "AIG", "AMAT", "AMD", 
               "AMGN", "AMZN", "AOCA", "ARCO", "ARKK", "ASR", "AUY", "AVGO", "AXP", "AZN", "BA", "BA.C", "BABA", "BB", 
               "BBD", "BBV", "BCS", "BHP", "BIDU", "BIIB", "BIOX", "BITF", "BK", "BMY", "BNG", "BP", "BRFS", "BRKB", "BSBR", 
               "C", "CAAP", "CAH", "CAR", "CAT", "CBRD", "CDE", "CL", "COIN", "COST", "CRM", "CS", "CSCO", "CVX", "CX", "DD", 
               "DE", "DESP", "DIA", "DISN", "DOCU", "DOW", "E", "EA", "EBAY", "EEM", "EFX", "ERIC", "ERJ", "ETSY", "EWZ", "F", 
               "FCX", "FDX", "FMX", "FSLR", "GE", "GFI", "GGB", "GILD", "GLOB", "GLW", "GM", "GOLD", "GOOGL", "GPRK", "GRMN", 
               "GS", "HAL", "HD", "HL", "HMC", "HMY", "HOG", "HON", "HPQ", "HSBC", "HSY", "HUT", "HWM", "IBM", "IFF", "INTC", 
               "ITUB", "IWM", "JD", "JMIA", "JNJ", "JPM", "KMB", "KO", "KOFM", "LLY", "LMT", "LRCX", "LVS", "LYG", "MA", "MCD", 
               "MDT", "MELI", "META", "MMM", "MO", "MOS", "MRK", "MSFT", "MSI", "MSTR", "MU", "NEM", "NFLX", "NGG", "NIO", "NKE", 
               "NOKA", "NTCO", "NTES", "NUE", "NVDA", "NVS", "ORAN", "ORCL", "OXY", "PAAS", "PAC", "PANW", "PBI", "PBR", "PCAR", 
               "PEP", "PFE", "PG", "PHG", "PKS", "PSX", "PYPL", "QCOM", "QQQ", "RBLX", "RIO", "RTX", "SAN", "SAP", "SATL", "SBUX", 
               "SCCO", "SE", "SHEL", "SHOP", "SI", "SID", "SLB", "SNAP", "SNOW", "SONY", "SPGI", "SPOT", "SPY", "SQ", "SYY", "T", 
               "TEFO", "TEN", "TGT", "TM", "TMO", "TRIP", "TRVV", "TSLA", "TSM", "TTE", "TV", "TWLO", "TXN", "TXR", "UAL", "UBER", 
               "UGP", "UL", "UNH", "UNP", "UPST", "USB", "V", "VALE", "VIST", "VIV", "VOD", "VZ", "WBA", "WFC", "WMT", "X", "XLE", 
               "XLF", "XOM", "XP", "YY", "ZM"]

    adrs = ["BBAR", "BMA", "CEPU", "CRESY", "EDN", "GGAL", "IRS", "LOMA", "PAM", "SUPV", "TEO", "TGS", "TS", "TX", "YPF"]

    sectors = ["XLC", "XLP", "XLY", "XLF", "XLV", "XLI", "XLRE", "XLU", "XBI", "XLB", "XLK", "XLE"]
    
    precarga = ["lideres", "general", "cedears", "adrs", "sectores"]
    precarga_dict = {"lideres" : lideres_arg, "general" : general_arg, "cedears" : cedears, "adrs" : adrs, "sectores" : sectors}
    
    if activos in precarga:
        activos = precarga_dict[activos]
    
    if tipo == "no end":
        data = yf.download(activos, start = data_from, interval = interval, progress = False, auto_adjust = True)
    elif tipo == "end":
        data = yf.download(activos, start = data_from, end = data_to, interval = interval, progress = False, auto_adjust = True)
    elif tipo == "period":
        data = yf.download(activos, interval = interval, period= period, progress = False, auto_adjust = True)
    
    if swap:
        #data = data.swaplevel(i = 0, j = 1, axis = 1)
        # Algoritmo para procesar el MultipleTicker download de yfinance
        dicto = {}
        low = data["Low"]
        high = data["High"]
        close = data["Close"]
        open = data["Open"]
        volume = data["Volume"]

        tickers = list(data["Close"].columns)

        for ticker in tickers:
            dicto[ticker] = {
                "Open" : open[ticker],
                "High" : high[ticker],
                "Low" : low[ticker],
                "Close" : close[ticker],
                "Volume" : volume[ticker]
            }

            dicto[ticker] = pd.DataFrame(dicto[ticker])
        return dicto
    return data

In [3]:
# FUNCIONALIDADES PRINCIPALES DEL MÓDULO:

def filterTrend(activos, timeframe, report = False):
    """
    Esta función utiliza el indicador técnico ADX para determinar la fortaleza de la tendencia. Adicionalmente se informa el sentido
    de la tendencia según el sentido de los cruces de los DMI- y DMI+. Si DMI+ > DMI - => Tendencia Alcista, sino Bajista.
    Se utilizan los siguientes rangos: 
        - Sin tendencia:                    0  >= ADX < 25
        - tendencia Fuerte:                 25 >= ADX < 50
        - tendencia Muy Fuerte              50 >= ADX < 75
        - tendencia Extremadamente fuerte   75 >= ADX < 100
                       
    # Inputs:
        - activos: es una lista de activos a analizar. Ej.: adrs = ["GGAL", "BBAR", "LOMA", "CEPU"]. También hay sets de activos preconfigurados en la función:
            - "lideres": si se desea cargar todo el panel lider de Argentina;
            - "general": si se desea cargar todo el panel general de Argentina;
            - "adrs": si se desea cargar todo el listado de ADRs argentinos (en usd);
            - "sectors": si se desea cargar el listado de sectores de la economía (ETFs representativos de USA);
            - "cedears": si se desea cargar todo el listado de certificados de depósito de activos del exterior que cotizan en Argentina (puede demorar bastante).
        - timeframe: es la duración de cada sesión/vela (compresión temporal). Puede ser "diario", "semanal" o "mensual".
        - report: si se pasa "True" como argumento, genera un archivo excel en el directorio del script con la tabla estilada.
        
    # Outputs:
        - tabla_final_activos_en_filas: es un DF resumen con los activos en las filas y las variables en las columnas.
        - tabla_final_activos_en_columnas: es un DF resumen con los activos en las columnas y las variables en las filas.
        - estilado: tabla_final_activos_en_filas estilada con colores rojos y verdes según tendencia y un gradient en el ADX.
        - screener: devuelve un diccionario con los listados de activos que están en los distintos rangos de tendencia (clasificación)
    """
    
    import pandas as pd
    import pandas_ta as ta
    import yfinance as yf
    
    if timeframe == "diario":
        interval = "1d"
        period = "1y"
    elif timeframe == "semanal":
        interval = "1wk"
        period = "2y"
    elif timeframe == "mensual":
        interval = "1mo"
        period = "4y"
    else:
        return "timeframe incorrecto, elegir 'diario', 'semanal' o 'mensual'"
    
    data = getDataYfMulti(activos, tipo = "period", interval = interval, period = period, swap = True)
    
    # Defino datos de estrategia  # Features: ADX_14, DMP_14 y DMN_14
    MyStrategy = ta.Strategy(
    name = "describirTendencia",
    ta = [
        {"kind" : "adx"},
        ]
    )

    # Creamos DFs para operadores:
    #df_adx = pd.DataFrame(index = data.index)  # Es el feature numerico (ADX_14)
    #df_tendencia = pd.DataFrame(index = data.index) # Es el feature categórico: "Fuerte", "Media", "Sin tendencia"
  
    dict_resumenPorActivo = {}  # ==> Resumen por activo (un diccionario de diccionarios).

    i = 0
    
    tickers = list(data.keys())
    
    for activo in tickers:
        try:
            datat = pd.DataFrame(data[activo])
            datat.ta.strategy(MyStrategy)
            datat.dropna(inplace = True)
            datat["tendencia"] = np.where(datat["ADX_14"] < 25, "Sin tendencia", 
                                          np.where(((datat["ADX_14"] >= 25) & (datat["ADX_14"] < 50)), "Fuerte",
                                                np.where(((datat["ADX_14"] >= 50) & (datat["ADX_14"] < 75)), "Muy Fuerte",
                                                         np.where(datat["ADX_14"] >= 75 , "Extrem Fuerte", ""))))
            datat["sentido"] = np.where(datat["DMP_14"] >= datat["DMN_14"], "Alcista", "Bajista")
                                          
                      
            # Cargamos el activo:
            dict_resumenPorActivo[activo] = {"tendencia" : datat["tendencia"][-1],
                                             "adx" : datat["ADX_14"][-1],
                                             "sentido" : datat["sentido"][-1],
                                             }
            i += 1
            print(f"[{activo}]Procesando {i} de {len(tickers)} activos ...")

        except:
            pass
    
    tabla_final_activos_en_columnas = pd.DataFrame(dict_resumenPorActivo)
    tabla_final_activos_en_filas = tabla_final_activos_en_columnas.transpose()
    
    tabla = tabla_final_activos_en_filas
    
    def set_styles(tabla):
        def highlight_sentido(val, color_if_true, color_if_false):
            color = color_if_true if val == "Alcista" else color_if_false
            return f"background-color: {color}; color: black"
        def highlight_tend(val, color_if_true):
            color = color_if_true if val == "Sin tendencia" else None
            return f"background-color: {color}"
                

        styler = tabla.style.applymap(highlight_sentido, color_if_true='#90EF90', color_if_false='#FA6B84', 
                              subset=['sentido']) \
                            .applymap(highlight_tend, color_if_true='grey', 
                              subset=['tendencia']) \
                            .applymap(lambda _: "font-weight: bold") \
                            .background_gradient(subset = ["adx"]) 
        
        return styler
    
    estilado = set_styles(tabla)
    
    # Generación de reporte en excel (opcional)
    if report == True:
        from datetime import date
        today = date.today()
        estilado.to_excel(f'reporteTendencias-{today}.xlsx')
    
    
    tendencia_extremfuerte = list(tabla_final_activos_en_filas.loc[tabla_final_activos_en_filas["tendencia"] == "Extrem Fuerte"].index)
    tendencia_muyfuerte = list(tabla_final_activos_en_filas.loc[tabla_final_activos_en_filas["tendencia"] == "Muy Fuerte"].index)
    tendencia_fuerte = list(tabla_final_activos_en_filas.loc[tabla_final_activos_en_filas["tendencia"] == "Fuerte"].index)
    tendencia_sin = list(tabla_final_activos_en_filas.loc[tabla_final_activos_en_filas["tendencia"] == "Sin tendencia"].index)
    
    alcistas_extremfuerte = list(tabla_final_activos_en_filas.loc[
        (tabla_final_activos_en_filas["tendencia"] == "Extrem Fuerte") & (tabla_final_activos_en_filas["sentido"] == "Alcista")].index)
    alcistas_muyfuerte = list(tabla_final_activos_en_filas.loc[
        (tabla_final_activos_en_filas["tendencia"] == "Muy Fuerte") & (tabla_final_activos_en_filas["sentido"] == "Alcista")].index)
    
    screener = {"tendencia_extremfuerte" : tendencia_extremfuerte, "tendencia_muyfuerte" : tendencia_muyfuerte,
               "tendencia_fuerte" : tendencia_fuerte, "tendencia_sin" : tendencia_sin,
               "alcistas_extremfuerte" : alcistas_extremfuerte, "alcistas_muyfuerte" : alcistas_muyfuerte}
    
    return (tabla_final_activos_en_filas, 
            tabla_final_activos_en_columnas, 
            estilado, 
            screener)



def getDMISignals(activos, timeframe):
    
    """
    Esta función devuelve los cruces vigentes de los indicadores DMI+ y DMI- del listado de activos
    solicitado, así como el sentido del cruce, la fecha de la señal, el precio de la señal, el rendimiento acumulado a la fecha
    y el rendimiento acumulado, todo ordenado en forma descendente por cantidad de días transcurridos desde la señal.
    
    # Inputs:
        - activos: es una lista de activos a analizar. Ej.: adrs = ["GGAL", "BBAR", "LOMA", "CEPU"]. También hay sets de activos preconfigurados en la función:
            - "lideres": si se desea cargar todo el panel lider de Argentina;
            - "general": si se desea cargar todo el panel general de Argentina;
            - "adrs": si se desea cargar todo el listado de ADRs argentinos (en usd);
            - "sectors": si se desea cargar el listado de sectores de la economía (ETFs representativos de USA);
            - "cedears": si se desea cargar todo el listado de certificados de depósito de activos del exterior que cotizan en Argentina (puede demorar bastante).
        - timeframe: es la duración de cada sesión/vela (compresión temporal). Puede ser "diario", "semanal" o "mensual".
    
    # Outputs:
        - dmis_cross_signals: diccionario con las señales vigentes;
        - dmis_cross_signals_df: tabla con las señales vigentes;
        - dmis_cross_signals_df_formateado: tabla con las señales vigentes formateada con escala de colores según rendimiento
        acumulado anualizado desde la señal.
        
    """
    import pandas as pd
    import pandas_ta as ta
    import yfinance as yf
    
    if timeframe == "diario":
        interval = "1d"
        period = "3y"
    elif timeframe == "semanal":
        interval = "1wk"
        period = "10y"
    elif timeframe == "mensual":
        interval = "1mo"
        period = "15y"
    else:
        return "timeframe incorrecto, elegir 'diario', 'semanal' o 'mensual'"
    
    data = getDataYfMulti(activos, tipo = "period", interval = interval, period = period, swap = True)

    # Defino datos de estrategia  # Features: ADX_14, DMP_14 y DMN_14
    MyStrategy = ta.Strategy(
    name = "describirTendencia",
    ta = [
        {"kind" : "adx"},
        ]
    )
 
    dmis_cross_signals = {}

    tickers = list(data.keys())
    
    for activo in tickers:
        try:
            # Iteramos por cada activo
            # Armamos el dataframe OHLC y calculamos componentes del indicador
            datat = pd.DataFrame(data[activo])  
            datat.ta.strategy(MyStrategy)
            datat.dropna(inplace = True)
            datat["distancia"] = (datat["DMP_14"] / datat["DMN_14"]) - 1

            # Obtenemos el precio actual
            actual_price = round(datat["Close"][-1],2)
            # Determinamos posición y cambios de los indicadores:
            datat["posicionDMIs"] = np.where(datat["distancia"] > 0, 1, -1)
            datat["posicionDMIs_change"] = np.where(datat["posicionDMIs"] > datat["posicionDMIs"].shift(1), "Compra", 
                                                        np.where(datat["posicionDMIs"] < datat["posicionDMIs"].shift(1), "Venta", ""))


            # 1) Caso de Long, calculamos estadísticas:
            if datat["posicionDMIs"][-1] > 0:
                last_buy_signal_crossover = datat[datat["posicionDMIs_change"] == "Compra"].iloc[-1]
                price_lbs_crossover = round(last_buy_signal_crossover["Close"],2)
                date_lbs_crossover = datetime(last_buy_signal_crossover.name.year, 
                                                    last_buy_signal_crossover.name.month, 
                                                    last_buy_signal_crossover.name.day)
                days_since_bs_crossover = (datetime.today() - date_lbs_crossover).days
                rend_since_bs_crossover = round(((actual_price / price_lbs_crossover) - 1) * 100, 2)
                rend_since_bs_annualized_crossover = round((((1 + rend_since_bs_crossover/100) ** (365/days_since_bs_crossover)) - 1) * 100,2)
                dmis_cross_signals[activo] = {"type" : "LONG", 
                                                        "precioSeñal" : price_lbs_crossover, 
                                                        "fechaSeñal": date_lbs_crossover.strftime("%Y-%m-%d"), 
                                                        "diasDesdeSeñal" : days_since_bs_crossover,
                                                        "precioActual" : round(actual_price,2),
                                                        "rendAcumDesdeSeñal":  rend_since_bs_crossover, 
                                                        "rendAnualDesdeSeñal" : rend_since_bs_annualized_crossover}


            # 2) Caso de Short, calculamos estadísticas:     
            elif datat["posicionDMIs"][-1] < 0:
                last_sell_signal_crossover = datat[datat["posicionDMIs_change"] == "Venta"].iloc[-1]
                price_lss_crossover = round(last_sell_signal_crossover["Close"],2)
                date_lss_crossover = datetime(last_sell_signal_crossover.name.year, 
                                                last_sell_signal_crossover.name.month, 
                                                last_sell_signal_crossover.name.day)
                days_since_ss_crossover = (datetime.today() - date_lss_crossover).days
                rend_since_ss_crossover = round(((price_lss_crossover / actual_price) - 1) * 100, 2)
                rend_since_ss_annualized_crossover = round((((1 + rend_since_ss_crossover/100) ** (365/days_since_bs_crossover)) - 1) * 100,2)
                dmis_cross_signals[activo] = {"type" : "SHORT", 
                                                "precioSeñal" : price_lss_crossover, 
                                                "fechaSeñal": date_lss_crossover.strftime("%Y-%m-%d"), 
                                                "diasDesdeSeñal" : days_since_ss_crossover,
                                                "precioActual" : round(actual_price,2),
                                                "rendAcumDesdeSeñal":  rend_since_ss_crossover, 
                                                "rendAnualDesdeSeñal" : rend_since_ss_annualized_crossover}
            print(f"{activo} - procesado!")
        except:
            print(f"{activo} - no pudo ser procesado")
            pass

    dmis_cross_signals_df = pd.DataFrame(dmis_cross_signals).transpose().sort_values(by = "diasDesdeSeñal", ascending = True)
    dmis_cross_signals_df_formateado = dmis_cross_signals_df.style.background_gradient(axis=0, gmap=dmis_cross_signals_df['rendAnualDesdeSeñal'], 
                                                                                         cmap='Greens') \
    .format('{:.2f}%', subset=["rendAcumDesdeSeñal", "rendAnualDesdeSeñal"]) \
    .format('{:.2f}', subset=["precioSeñal", "precioActual"])
    #.format('{:.2f}%', subset=["rendAcumDesdeSeñal", "rendAnualDesdeSeñal"]) \
    
    
    
    return dmis_cross_signals, dmis_cross_signals_df, dmis_cross_signals_df_formateado

In [4]:
tablax, _, estilada, screener = filterTrend("adrs", timeframe = "semanal", report = False)

[BBAR]Procesando 1 de 15 activos ...
[BMA]Procesando 2 de 15 activos ...
[CEPU]Procesando 3 de 15 activos ...
[CRESY]Procesando 4 de 15 activos ...
[EDN]Procesando 5 de 15 activos ...
[GGAL]Procesando 6 de 15 activos ...
[IRS]Procesando 7 de 15 activos ...
[LOMA]Procesando 8 de 15 activos ...
[PAM]Procesando 9 de 15 activos ...
[SUPV]Procesando 10 de 15 activos ...
[TEO]Procesando 11 de 15 activos ...
[TGS]Procesando 12 de 15 activos ...
[TS]Procesando 13 de 15 activos ...
[TX]Procesando 14 de 15 activos ...
[YPF]Procesando 15 de 15 activos ...


In [6]:
# filterTrend output tabla con formato
estilada

,tendencia,adx,sentido
BBAR,Sin tendencia,20.739608,Alcista
BMA,Fuerte,28.910379,Alcista
CEPU,Fuerte,31.319849,Alcista
CRESY,Fuerte,31.258108,Alcista
EDN,Fuerte,32.855815,Alcista
GGAL,Fuerte,25.806613,Alcista
IRS,Sin tendencia,24.934033,Alcista
LOMA,Sin tendencia,16.427822,Alcista
PAM,Sin tendencia,22.448086,Alcista
SUPV,Fuerte,33.825643,Alcista


In [7]:
# filterTrend output screener
screener

{'tendencia_extremfuerte': [],
 'tendencia_muyfuerte': [],
 'tendencia_fuerte': ['BMA', 'CEPU', 'CRESY', 'EDN', 'GGAL', 'SUPV', 'YPF'],
 'tendencia_sin': ['BBAR', 'IRS', 'LOMA', 'PAM', 'TEO', 'TGS', 'TS', 'TX'],
 'alcistas_extremfuerte': [],
 'alcistas_muyfuerte': []}

In [8]:
signals, signals_df, signals_estilado = getDMISignals(activos = "adrs", timeframe = "semanal")

BBAR - procesado!
BMA - procesado!
CEPU - procesado!
CRESY - procesado!
EDN - procesado!
GGAL - procesado!
IRS - procesado!
LOMA - procesado!
PAM - procesado!
SUPV - procesado!
TEO - procesado!
TGS - procesado!
TS - procesado!
TX - procesado!
YPF - procesado!


In [9]:
# getDMISignals output - signals
signals

{'BBAR': {'type': 'LONG',
  'precioSeñal': 5.24,
  'fechaSeñal': '2023-11-20',
  'diasDesdeSeñal': 64,
  'precioActual': 5.58,
  'rendAcumDesdeSeñal': 6.49,
  'rendAnualDesdeSeñal': 43.13},
 'BMA': {'type': 'LONG',
  'precioSeñal': 25.34,
  'fechaSeñal': '2023-11-20',
  'diasDesdeSeñal': 64,
  'precioActual': 30.22,
  'rendAcumDesdeSeñal': 19.26,
  'rendAnualDesdeSeñal': 173.06},
 'CEPU': {'type': 'LONG',
  'precioSeñal': 7.46,
  'fechaSeñal': '2023-11-20',
  'diasDesdeSeñal': 64,
  'precioActual': 9.29,
  'rendAcumDesdeSeñal': 24.53,
  'rendAnualDesdeSeñal': 249.43},
 'CRESY': {'type': 'LONG',
  'precioSeñal': 7.27,
  'fechaSeñal': '2023-10-09',
  'diasDesdeSeñal': 106,
  'precioActual': 9.28,
  'rendAcumDesdeSeñal': 27.65,
  'rendAnualDesdeSeñal': 131.78},
 'EDN': {'type': 'LONG',
  'precioSeñal': 16.4,
  'fechaSeñal': '2023-11-20',
  'diasDesdeSeñal': 64,
  'precioActual': 20.29,
  'rendAcumDesdeSeñal': 23.72,
  'rendAnualDesdeSeñal': 236.66},
 'GGAL': {'type': 'LONG',
  'precioSeña

In [10]:
# getDMISignals output - signals dataframe
signals_df

,type,precioSeñal,fechaSeñal,diasDesdeSeñal,precioActual,rendAcumDesdeSeñal,rendAnualDesdeSeñal
TX,SHORT,38.86,2024-01-15,8,38.65,0.54,3.12
TS,SHORT,32.31,2024-01-08,15,31.98,1.03,6.02
BBAR,LONG,5.24,2023-11-20,64,5.58,6.49,43.13
BMA,LONG,25.34,2023-11-20,64,30.22,19.26,173.06
CEPU,LONG,7.46,2023-11-20,64,9.29,24.53,249.43
EDN,LONG,16.4,2023-11-20,64,20.29,23.72,236.66
GGAL,LONG,15.32,2023-11-20,64,18.18,18.67,165.45
IRS,LONG,8.12,2023-11-20,64,8.69,7.02,47.25
LOMA,LONG,6.5,2023-11-20,64,7.32,12.62,96.96
PAM,LONG,45.59,2023-11-20,64,48.13,5.57,36.22


In [11]:
# getDMISignals output - signals dataframe con estilos
signals_estilado

,type,precioSeñal,fechaSeñal,diasDesdeSeñal,precioActual,rendAcumDesdeSeñal,rendAnualDesdeSeñal
TX,SHORT,38.86,2024-01-15,8,38.65,0.54%,3.12%
TS,SHORT,32.31,2024-01-08,15,31.98,1.03%,6.02%
BBAR,LONG,5.24,2023-11-20,64,5.58,6.49%,43.13%
BMA,LONG,25.34,2023-11-20,64,30.22,19.26%,173.06%
CEPU,LONG,7.46,2023-11-20,64,9.29,24.53%,249.43%
EDN,LONG,16.40,2023-11-20,64,20.29,23.72%,236.66%
GGAL,LONG,15.32,2023-11-20,64,18.18,18.67%,165.45%
IRS,LONG,8.12,2023-11-20,64,8.69,7.02%,47.25%
LOMA,LONG,6.50,2023-11-20,64,7.32,12.62%,96.96%
PAM,LONG,45.59,2023-11-20,64,48.13,5.57%,36.22%
